# **Tên dự án** : Dữ đoán thông số sản xuất và thiết kế ngành In vải bằng máy học 
---
**1. Mô tả dự án:**
* Loại dự án - Phân loại
* Mô tả dự án:
    * Ngành công nghiệp in vải đang phát triển mạnh mẽ, với nhu cầu ngày càng cao về chất lượng in ấn, độ chính xác của màu sắc và hiệu suất sản xuất. Một trong những thách thức lớn nhất của ngành là việc xác định chính xác các thông số sản xuất (nhiệt độ, tốc độ ép, loại máy in, ...) và thông số thiết kế (mã màu, ICC profile, ...) để đảm bảo chất lượng đầu ra tối ưu.
    * Hiện nay, việc xác định các thông số này chủ yếu dựa vào kinh nghiệm của nhân viên kỹ thuật. Quy trình thủ công này có nhiều hạn chế:
        - Tốn thời gian: Các thông số cần được điều chỉnh thủ công, dẫn đến việc phải thử nghiệm nhiều lần.
        - Không nhất quán: Kinh nghiệm của mỗi nhân viên khác nhau, dẫn đến chất lượng in ấn không đồng đều.
        - Tỷ lệ lỗi cao: Sai lệch về màu sắc so với bản thiết kế gốc có thể xảy ra, gây lãng phí nguyên vật liệu.
    * Với sự phát triển của Trí tuệ nhân tạo (AI) và Học máy (Machine Learning), việc ứng dụng các thuật toán học máy vào dự đoán thông số sản xuất có thể giúp tự động hóa quy trình, tăng độ chính xác và giảm chi phí sản xuất.
    * Dự án FabricML: Dự đoán Thông Số Sản Xuất và Thiết Kế Ngành In Vải Bằng Máy Học được thực hiện nhằm giải quyết bài toán này, giúp tối ưu hóa quy trình sản xuất thông qua công nghệ học máy.
    
**2. Chi tiết chính**
* Tập dữ liệu bao gồm các thông tin sau:
    - LOẠI VẢI: Chất liệu vải (lụa Latin, thun lạnh, chiffon, gấm, v.v.)
    - CMYK ĐO: Giá trị màu đo được từ thiết bị (định dạng C-M-Y-K, ví dụ: 50-12-5-30)
    - CMYK FILE: Giá trị màu mục tiêu cần đạt được (định dạng tương tự)
    - MÁY IN: Loại máy in được sử dụng (ATEXCO-1, ATEXCO-2, MÁY 18, v.v.)
    - ICC: Hồ sơ màu (VEGAPRINT DEFAULT, PRINTFACTORY 2024, RIIN DEFAULT, v.v.)
    - MÁY ÉP: Thông số máy ép (ví dụ: 4.0, 5.0, 6.0)
    - TỐC ĐỘ: Tốc độ in (m/min)
    - HIỆT ĐỘ: Nhiệt độ ép (℃)
* Dữ liệu đầu vào:
    - Thông số CMYK đo được (CMYK ĐO): Gồm 4 giá trị C (Cyan), M (Magenta), Y (Yellow), K (Key/Black) được đo từ mẫu vải
    - Loại vải: Các loại vải khác nhau như LỤA LATIN, THUN LẠNH, CHIFON,...
    - Thông số máy móc: MÁY IN, ICC (hồ sơ màu), MÁY ÉP
* Mục tiêu dự đoán:
    - Thông số CMYK FILE (Thông số sản xuất lịch sử từ các lần in trước.): 4 giá trị C, M, Y, K của file thiết kế
    - Các thông số vận hành: TỐC ĐỘ, NHIỆT ĐỘ

## 1. Nhập thư viện 
---
Ở đây đang nhập tất cả các thư viện cần thiết.

In [2]:
# Nhập thư viện
# Nhập Numpy & Pandas để xử lý dữ liệu & sắp xếp dữ liệu
import numpy as np
import pandas as pd

# Nhập công cụ để trực quan hóa
import matplotlib.pyplot as plt
import seaborn as sns

# Nhập thư viện số liệu đánh giá
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
#from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report


# Thư viện được sử dụng để xử lý trước dữ liệu
from sklearn.preprocessing import LabelEncoder

# Nhập thư viện lựa chọn mô hình
from sklearn.model_selection import train_test_split, GridSearchCV

# Thư viện được sử dụng để triển khai Mô hình ML
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Thư viện được sử dụng để bỏ qua cảnh báo
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


**Nhập bộ dữ liệu**

* Bây giờ hãy tải tập dữ liệu vào khung dữ liệu của gấu trúc. Để xử lý dữ liệu tốt hơn và tận dụng các chức năng tiện dụng để thực hiện các tác vụ phức tạp trong một lần.

In [6]:
# Tải tập dữ liệu
df = pd.read_excel("FAKE_DATA_1000.xlsx")

**Bộ dữ liệu đầu tiên**

In [ ]:
# Xem 5 hàng đầu tiên của tập dữ liệu
df.head()

**Số lượng hàng và cột của tập dữ liệu**

In [76]:
# Đếm số hàng và cột của tập dữ liệu
# Kiểm tra số hàng và cột của tập dữ liệu bằng hình dạng
print("Số hàng: ", df.shape[0])
print("Số cột: ", df.shape[1])

Số hàng:  1010
Số cột:  8


**Thông tin tập dữ liệu**

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   LOẠI VẢI   1010 non-null   object
 1   CMYK ĐO    1010 non-null   object
 2   CMYK FILE  1010 non-null   object
 3   MÁY IN     1010 non-null   object
 4   ICC        1010 non-null   object
 5   MÁY ÉP     1010 non-null   int64 
 6   TỐC ĐỘ     1010 non-null   int64 
 7   NHIỆT ĐỘ   1010 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 63.3+ KB


**Kiểm tra Giá trị trùng lặp**

In [ ]:
dup = df.duplicated().sum()
print(f'Số hàng trùng lặp: {dup}')

**Giá trị bị thiếu/Giá trị Null**

In [80]:
df.isnull().sum()

LOẠI VẢI     0
CMYK ĐO      0
CMYK FILE    0
MÁY IN       0
ICC          0
MÁY ÉP       0
TỐC ĐỘ       0
NHIỆT ĐỘ     0
dtype: int64

ádasd

## 2. Hiểu các biến
---

In [84]:
# Cột dữ liệu
df.columns

Index(['LOẠI VẢI', 'CMYK ĐO', 'CMYK FILE', 'MÁY IN', 'ICC', 'MÁY ÉP', 'TỐC ĐỘ',
       'NHIỆT ĐỘ'],
      dtype='object')

In [86]:
# Mô tả bộ dữ liệu (bao gồm tất cả các cột)
df.describe(include='all').round(2)

,LOẠI VẢI,CMYK ĐO,CMYK FILE,MÁY IN,ICC,MÁY ÉP,TỐC ĐỘ,NHIỆT ĐỘ
count,1010,1010,1010,1010,1010,1010.00,1010.00,1010.0
unique,10,1010,1010,7,7,NaN,NaN,NaN
top,LỤA LATIN,50-12-5-30,48-10-7-32,ATEXCO-1,PRINTFACTORY DEFAULT,NaN,NaN,NaN
freq,119,1,1,513,511,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,5.86,11.94,225.0
std,NaN,NaN,NaN,NaN,NaN,0.62,4.07,0.0
min,NaN,NaN,NaN,NaN,NaN,1.00,10.00,225.0
25%,NaN,NaN,NaN,NaN,NaN,6.00,10.00,225.0
50%,NaN,NaN,NaN,NaN,NaN,6.00,11.00,225.0
75%,NaN,NaN,NaN,NaN,NaN,6.00,12.00,225.0


**Kiểm tra Giá trị duy nhất cho mỗi biến.**

In [45]:
for i in df.columns.tolist():
    print("Số giá trị duy nhất trong", i, "là", df[i].nunique())

Số giá trị duy nhất trong LOẠI VẢI là 10
Số giá trị duy nhất trong CMYK ĐO là 1010
Số giá trị duy nhất trong CMYK FILE là 1010
Số giá trị duy nhất trong MÁY IN là 7
Số giá trị duy nhất trong ICC là 7
Số giá trị duy nhất trong MÁY ÉP là 6
Số giá trị duy nhất trong TỐC ĐỘ là 26
Số giá trị duy nhất trong NHIỆT ĐỘ là 1


## 3. Trực quan hóa dữ liệu ngành in vải
---
Dưới đây là các biểu đồ trực quan hóa quan trọng giúp phân tích dữ liệu in vải:


### 3.1 Phân bố các loại vải

Nhận xét: Dữ liệu có sự cân bằng tương đối giữa các loại vải, với Lụa Latin và Thun lạnh chiếm tỷ lệ cao nhất.

### 3.2. Biểu đồ nhiệt tương quan

Xem xét tương quan giữa các biến:

### 3.3 Giá trị thực tế vs dự đoán cho từng mô hình

Chúng ta sẽ sửa đổi vòng lặp chính để lưu trữ dự đoán và sau đó trực quan hóa chúng:

## 4. Xử lý dữ liệu
---

### 4.1 Tách giá trị CMYK từ chuỗi
---
Tách cột CMYK thành 4 cột riêng biệt

Cột CMYK ĐO và CMYK FILE chứa dữ liệu dạng chuỗi số, ví dụ "50-12-5-30". Chúng được tách thành 4 cột riêng biệt (C, M, Y, K) và chuyển đổi sang kiểu số (float).

In [8]:
df_cmyk_do = df["CMYK ĐO"].str.split('-', expand=True).astype(float)
df_cmyk_do.columns = ["CMYK ĐO_C", "CMYK ĐO_M", "CMYK ĐO_Y", "CMYK ĐO_K"]

df_cmyk_file = df["CMYK FILE"].str.split('-', expand=True).astype(float)
df_cmyk_file.columns = ["CMYK FILE_C", "CMYK FILE_M", "CMYK FILE_Y", "CMYK FILE_K"]


### 4.2 Mã hóa biến phân loại

* Các cột chứa dữ liệu dạng chữ (LOẠI VẢI, MÁY IN, ICC, MÁY ÉP) được chuyển thành số bằng LabelEncoder.

In [10]:
categorical_cols = ["LOẠI VẢI", "MÁY IN", "ICC", "MÁY ÉP"]
df_cat_transformed = df[categorical_cols].copy()
label_encoders = {} #  Khởi tạo dictionary giúp lưu lại bộ mã hóa

for col in categorical_cols:
    le = LabelEncoder()
    df_cat_transformed[col] = le.fit_transform(df_cat_transformed[col])# mã hoá dữ liệu
    label_encoders[col] = le  # Lưu lại encoder nếu cần dùng sau


* Chuyển đổi biến phân loại (LOẠI VẢI, MÁY IN, ICC, MÁY ÉP) sang dạng số bằng LabelEncoder().
* Điều này cần thiết vì mô hình XGBoost không xử lý dữ liệu dạng chuỗi.

### 4.3. Chuẩn hóa dữ liệu số
---
* Chuẩn hóa dữ liệu số bằng StandardScaler
* Các cột TỐC ĐỘ và NHIỆT ĐỘ được chuẩn hóa để dữ liệu có phân phối chuẩn.

In [13]:
numeric_cols = ["TỐC ĐỘ", "NHIỆT ĐỘ"]
df_num_transformed = df[numeric_cols].copy()
scaler_num = StandardScaler()
df_num_transformed[numeric_cols] = scaler_num.fit_transform(df_num_transformed[numeric_cols])

* Chuẩn hóa TỐC ĐỘ và NHIỆT ĐỘ để mô hình không bị ảnh hưởng bởi đơn vị đo lường.

### 4.4. Gộp tất cả lại thành DataFrame hoàn chỉnh

Các cột sau khi tiền xử lý được kết hợp lại (pd.concat), tạo ra tập dữ liệu sạch và sẵn sàng để đưa vào mô hình.

In [64]:
df_processed = pd.concat([df_cat_transformed, df_cmyk_do, df_cmyk_file, df_num_transformed], axis=1)
type(df_processed)

pandas.core.frame.DataFrame

* Gộp tất cả các dữ liệu đã xử lý để chuẩn bị cho việc huấn luyện mô hình.

## 5. Xây dựng mô hình
---

### 5.1 Chia dữ liệu thành đầu vào (X) và đầu ra (y)
* X: Các đặc trưng đầu vào như LOẠI VẢI, CMYK ĐO_C, CMYK ĐO_M, CMYK ĐO_Y, CMYK ĐO_K.
* y: Các giá trị cần dự đoán như CMYK FILE_C, CMYK FILE_M, CMYK FILE_Y, CMYK FILE_K, MÁY IN, ICC, MÁY ÉP, TỐC ĐỘ, NHIỆT ĐỘ.

In [17]:
input_columns = ["LOẠI VẢI", "CMYK ĐO_C", "CMYK ĐO_M", "CMYK ĐO_Y", "CMYK ĐO_K"]
target_columns = ["CMYK FILE_C", "CMYK FILE_M", "CMYK FILE_Y", "CMYK FILE_K", "MÁY IN", "ICC", "MÁY ÉP", "TỐC ĐỘ", "NHIỆT ĐỘ"]


In [19]:
# chia dữ liệu
X = df_processed[input_columns].copy()
y = df_processed[target_columns].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train.values
X_test = X_test.values


* Chia dữ liệu thành tập huấn luyện (70%) và tập kiểm tra (30%).

### 5.2 Danh sách mô hình huốn luyện 
* Ba mô hình Machine Learning được chọn:
    - Linear Regression: Mô hình tuyến tính đơn giản, dùng để kiểm tra mối quan hệ tuyến tính giữa các biến.
    - RandomForestRegressor: Mô hình học tập dựa trên nhiều cây quyết định, giúp dự đoán tốt hơn so với Linear Regression.
    - XGBoost: Một mô hình nâng cao, giúp cải thiện độ chính xác bằng cách giảm lỗi.
* Mỗi mô hình có bộ tham số (param_grid) dùng để tìm tham số tối ưu qua GridSearchCV.

#### 5.2.1 Huấn luyện mô hình XGBoost


In [27]:
model_xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
model_xgb.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

* Sử dụng XGBRegressor, một mô hình mạnh mẽ cho bài toán hồi quy.
* objective='reg:squarederror' được dùng cho bài toán hồi quy.

#### 5.2.2 Huấn luyện mô hình RandomForestRegressor

In [23]:
model_RandomForest = RandomForestRegressor(random_state=42)
model_RandomForest.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

#### 5.2.3 huấn luyện mô hình LinearRegression

In [25]:
model_LinearRegression = LinearRegression()
model_LinearRegression.fit(X_train, y_train)

LinearRegression()

### 5.3 Phân tích hiệu suất mô hình

In [29]:
print("XGBoost Accuracy:", model_xgb.score(X_test, y_test) * 100)
print("RandomForest Accuracy:", model_RandomForest.score(X_test, y_test) * 100)
print("LinearRegression Accuracy:", model_LinearRegression.score(X_test, y_test) * 100)

XGBoost Accuracy: 70.41963934898376
RandomForest Accuracy: 67.59324824001776
LinearRegression Accuracy: 63.18015084385134


### 5.4 Đánh giá mô hình

In [36]:
models = {
    "XGBoost": model_xgb,
    "Random Forest": model_RandomForest,
    "Linear Regression": model_LinearRegression
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)  # Tính R-squared

    print(f"{name}:")
    print(f"  MAE: {mae}")
    print(f"  MSE: {mse}")
    print(f"  RMSE: {rmse}")
    print(f"  R²: {r2}")
    print("-" * 40)

XGBoost:
  MAE: 0.43143290281295776
  MSE: 0.8425724506378174
  RMSE: 0.9179174530630831
  R²: 0.7041963934898376
----------------------------------------
Random Forest:
  MAE: 2.1776775746100037
  MSE: 15.402122776649126
  RMSE: 3.924553831539214
  R²: 0.6759324824001776
----------------------------------------
Linear Regression:
  MAE: 0.6906521077812617
  MSE: 1.828158682753215
  RMSE: 1.3520941841281675
  R²: 0.6318015084385133
----------------------------------------


* Kết quả của từng mô hình được đánh giá bằng:
    - MAE (Mean Absolute Error): Càng nhỏ càng tốt.
    - MSE (Mean Squared Error) & RMSE: Đo lường sai số, RMSE dễ hiểu hơn vì cùng đơn vị với dữ liệu thực tế.
    - R² (R-squared): Đánh giá mức độ mô hình giải thích được dữ liệu.

## 6. thử trên dữ liệu mẫu

In [119]:
# Dữ liệu test mẫu
test_sample = {
    "LOẠI VẢI": "CHIFON",
    "CMYK ĐO_C": 50,
    "CMYK ĐO_M": 40,
    "CMYK ĐO_Y": 30,
    "CMYK ĐO_K": 10
}

# Chuyển thành DataFrame
df_test = pd.DataFrame([test_sample])

In [121]:
# Mã hóa cột 'LOẠI VẢI' bằng LabelEncoder đã lưu
df_test["LOẠI VẢI"] = label_encoders["LOẠI VẢI"].transform(df_test["LOẠI VẢI"])

In [123]:
X_test_sample = df_test.values


In [134]:
y_pred_xgb = model_xgb.predict(X_test_sample)
y_pred_rf = model_RandomForest.predict(X_test_sample)
y_pred_lr = model_LinearRegression.predict(X_test_sample)

print("Dự đoán bằng XGBoost:", y_pred_xgb)
print("Dự đoán bằng Random Forest:", y_pred_rf)
print("Dự đoán bằng Linear Regression:", y_pred_lr)


Dự đoán bằng XGBoost: [[4.6420773e+01 3.3978172e+01 2.8946436e+01 8.9720869e+00 2.4794631e+00
  3.1524410e+00 5.0000525e+00 2.7404743e-01 7.1776640e-15]]
Dự đoán bằng Random Forest: [[43.22       29.87       29.75       10.33        2.47        4.47
   4.91        0.27596689  0.        ]]
Dự đoán bằng Linear Regression: [[46.55188305 34.39180705 28.84890531  9.59464928  2.70852794  4.15194441
   4.78503399  0.12831285  0.        ]]


In [138]:
# Chuyển kết quả dự đoán thành DataFrame
y_pred_df = pd.DataFrame(y_pred_xgb, columns=target_columns)

# Giải mã các cột phân loại
for col in ["MÁY IN", "ICC", "MÁY ÉP"]:
    y_pred_df[col] = label_encoders[col].inverse_transform(y_pred_df[col].astype(int))

# Giải mã các cột số liệu (chuẩn hóa ngược)
y_pred_df[["TỐC ĐỘ", "NHIỆT ĐỘ"]] = scaler_num.inverse_transform(y_pred_df[["TỐC ĐỘ", "NHIỆT ĐỘ"]])


print("Kết quả dự đoán sau khi chuyển đổi về dạng gốc:")
print(y_pred_df)

Kết quả dự đoán sau khi chuyển đổi về dạng gốc:
   CMYK FILE_C  CMYK FILE_M  CMYK FILE_Y  CMYK FILE_K    MÁY IN  \
0    46.420773    33.978172    28.946436     8.972087  ATEXCO-3   

                    ICC  MÁY ÉP     TỐC ĐỘ  NHIỆT ĐỘ  
0  PRINTFACTORY DEFAULT       6  13.052187     225.0  
